# GEOG0027  Google Earth Engine (GEE) for Shenzhen


This practical will use Google Earth Engine (GEE)'s python library [EE](https://developers.google.com/earth-engine) and [geemap library](https://geemap.org/). We will be applying the techniques and setup used previously in the course to the Shenzhen region. Make sure the prior lesson [introducing GEE](https://github.com/UCL-EO/GEOG0027/blob/main/docs/Intro_to_GEE.ipynb) has been completed.

After the first GEE practical lesson you will be able to use UCL's [Jupyter Hub](https://jupyter.data-science.rc.ucl.ac.uk/), and from there open a `Terminal` and use the following command to get the required files. Then open this notebook in jupyterhub to continue.

`wget https://github.com/UCL-EO/GEOG0027_Coursework/raw/main/docs/5a_PythonGEE-Shenzhen.ipynb'


## First map
For the Shenzhen classification work, we start from displaying a basemap of the area. The first time when you use GEE, you will need to have a Google account and provide an authorization code as instructed.

In [1]:
import geemap, ee, os, numpy
import ipyleaflet

Map = geemap.Map(center=[22.634, 114.19], zoom=9)
Map

Map(center=[22.634, 114.19], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

A basic Google Map-like interface should be displayed here now. If you can't see anything, please ensure that the ipyleaflet nbextension has been enabled.

## Examining the time series
Let's define a rectangular region of interest, following [min lon, min lat, max mon, max lat] first, and display a short 'movie' (a .gif file in fact) of how this area has changed over the past decades.

In [2]:
shenzhen_rec = ee.Geometry.Rectangle([113.7659, 22.40, 114.6654, 22.8536]) 

In [3]:
Map_gif = geemap.Map(center=[22.7511, 113.91], zoom=10)
Map_gif.add_landsat_ts_gif(roi=shenzhen_rec, start_year=1985, bands=['NIR', 'Red', 'Green'], frames_per_second=5)
Map_gif

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/h/Downloads/landsat_ts_jdt.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


Map(center=[22.7511, 113.91], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

The output map 'Map_gif' should look like something below.

![shenzhengif](images/landsat_ts_sgn.gif "shenzhen")

Next, we will compare such change by using a slider

In [15]:
landsat_ts = geemap.landsat_timeseries(roi=shenzhen_rec, start_year=1986, end_year=2020, \
                                       start_date='01-01', end_date='12-31')

layer_names = ['Landsat ' + str(year) for year in range(1986, 2021)]

geemap_landsat_vis = {
    'min': 0,
    'max': 3000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']} # You can change the vis bands here

Map2 = geemap.Map()
Map2.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, \
                 left_vis=geemap_landsat_vis, right_vis=geemap_landsat_vis)
Map2.centerObject(shenzhen_rec, zoom=10)
Map2

Map(center=[22.627302103068118, 114.2156500000001], controls=(WidgetControl(options=['position', 'transparent_…

We have previously defined a rectangle for Shenzhen area by coordinates, but we can also use existing shape/vector files to select Areas of Interest. 

## Select 'Shenzhen' as the area of interest (AOI)
The vector border layer is imported from https://developers.google.com/earth-engine/datasets/tags/borders, which includes the [Global Administrative Unit Layers (GAUL) data](https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level2) from 2015. You may notice that Shenzhen's boundary has expanded since (e.g. coastal landfill). We can, for example, manually draw/define another polygon and clip it to the GAUL border file, or, as a simple example, we will add a 'buffer' (e.g. 3000 meters) to the GAUL boundary data. This inevitably will introduce some areas outside the border of Shenzhen, e.g. part of Hong Kong, so you can work out some more elegant way of combining/clipping multiple AOI layers if time allows. Please justify your choice and summarize how you made it in the report. In the following example, I will use the `shenzhen_buffer` as my AOI.

In [37]:
cities = ee.FeatureCollection("FAO/GAUL/2015/level2")
#Map.addLayer(cities, {}, 'Cities', False)

shenzhen = cities.filter(ee.Filter.eq('ADM2_NAME', 'Shenzhen'))
outline = ee.Image().byte().paint(**{
  'featureCollection': shenzhen,
  'color': 1,
  'width': 3
})
Map.addLayer(outline, {}, 'Shenzhen')

# Next, add some buffer to include the coastal expansion areas
shenzhen_buffer = ee.Geometry(shenzhen.geometry()).buffer(3000)
Map.addLayer(shenzhen_buffer, {}, 'Buffer around Shenzhen')
#Map.addLayer(rec, "Original rec bounds")
Map

Map(bottom=57347.0, center=[22.634, 114.19], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

## Load Landsat data collections from GEE
Now we can see the the buffered AOI displayed on the Map. Next, let's load some Landsat images for the Shenzhen area. I've defined here a python function called `display_landsat_collection` to do so. It automatically loads both the [surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) and [annual NDVI](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NDVI) image collections from GEE's data catalog and also calculates the annual means for each band. 

You can skip most of the details of what's inside the code cell, but only to look at the first (and last) line of code. In order to run such function, you will need to choose a year (any year since 1984) and an AOI. In the following example, I choose year 2019 and the Shenzhen buffer to demonstrate the use of the code.

In [38]:
landsat_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': ['NIR', 'Red', 'Green']  # False Colour Composit bands to be visualised 
}
ndvi_colorized_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301']
}

In [39]:
def load_landsat_collection(Map,year, aoi, cloud_tolerance = 3.0, 
                            DISPLAY_ON_MAP = False, MEDIAN_ONLY = False):
    '''This function allows GEE to display a Landsat data collection 
    from any year between 1984 and present year
    that fall within the AOI and cloud tolerance, e.g. 3.0%.
    There are two optional flag:
    When DISPLAY_ON_MAP is TRUE, display this layer onto Map;
    When return_series = 'MEDIAN_ONLY', only median SR is loaded into landsat_ts, and
    Setting this option to MEDIAN_ONLY would be faster than loading other collections. 
    '''
    assert year >= 1984
    
    def renameBandsETM(image):
        if year >=1984: #LS8
            bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #, 'pixel_qa'
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'] #, 'pixel_qa'
#             return image.select(bands).rename(new_bands)
        elif year <=1984:
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
#         else: 
#             [B1, B2, B3, B4, B5, B6, B7]
        return image.select(bands).rename(new_bands)
        
    if not(MEDIAN_ONLY):
        if year >= 2013:
            layer_name = 'LC08' # LS8: 2013-now        
        elif year == 2012: # # LS7: 1999- , however SLC error >= 1999:
            layer_name = 'LE07' 
        elif year >=1984:
            layer_name = 'LT05' # LS5: 1984-2012
       
        collection_name_sr = f"LANDSAT/{layer_name}/C01/T1_SR" 
        # You can also use the following line, if interested in incorperating ndvi
        collection_name_ndvi = f"LANDSAT/{layer_name}/C01/T1_ANNUAL_NDVI" 

        all_sr_image = ee.ImageCollection(collection_name_sr) \
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_tolerance))\
            .sort('system:time_start') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER')

        all_sr_image = all_sr_image.map(renameBandsETM) # rename bands with 'renameBandsETM' function
        
        # reduce all_sr_image to annual average per pixel
        mean_image = all_sr_image.mean()
        mean_image = mean_image.clip(aoi).unmask()

        ndvi_image = ee.ImageCollection(collection_name_ndvi)\
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31')\
            .select('NDVI')\
            .first()
        ndvi_image = ndvi_image.clip(aoi).unmask()

        #mean_image.addBands(ndvi_image, 'NDVI')
    
    # This line loads all annual median surface ref
    landsat_ts = geemap.landsat_timeseries(roi=aoi, start_year=year, end_year=year, \
                                       start_date='01-01', end_date='12-31')

    median_image = landsat_ts.first().clip(aoi).unmask()
    
    if type(DISPLAY_ON_MAP) == geemap.geemap.Map:
        
        if not(MEDIAN_ONLY):
            Map.addLayer(ndvi_image, ndvi_colorized_vis, 'NDVI '+str(year),  opacity=0.9)
            Map.addLayer(mean_image, landsat_vis_param, "Mean Ref "+str(year))
        Map.addLayer(median_image, landsat_vis_param, "Median Ref "+str(year))

    if MEDIAN_ONLY:
        return median_image
    else:
        return all_sr_image, mean_image, median_image, ndvi_image 

Now the `load_landsat_collection` function has been defined, and we will run/execute it by calling the function name with appropriate input parameters (or 'arguments). The output of such function will be returned to the variables on the LHS of the equal sign, i.e. all_image_2019, mean_2019, median_2019, and ndvi_2019 in this case.

In [40]:
Map = geemap.Map(center=[22.634, 114.19], zoom=10)

# All you need to modify here is the YEAR below:
all_image, mean, median, ndvi= load_landsat_collection(Map,2019,\
                                        shenzhen_buffer, cloud_tolerance = 3,\
                                        DISPLAY_ON_MAP = Map)
Map

Map(bottom=57347.0, center=[22.634, 114.19], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

You should examine the functions under the `toolbar` and `layer` buttons on the top-right corner of the Map, e.g. use the `inspector` and `plotting` tools to check the data values, or use `layers` control to switch on/off layers or to adjust opacity.

We can also check the metadata from the Landsat image collection we just loaded from the Cloud. Have a look of the output. Any useful information?

In [41]:
first_image = all_image.first() 

props = geemap.image_props(first_image)
print( props.getInfo())
#print(first_image.get('IMAGE_DATE').getInfo())
#print(first_image.get('CLOUD_COVER').getInfo(), '%')

{'CLOUD_COVER': 0.21, 'CLOUD_COVER_LAND': 0.23, 'EARTH_SUN_DISTANCE': 0.989463, 'ESPA_VERSION': '2_23_0_1b', 'GEOMETRIC_RMSE_MODEL': 7.889, 'GEOMETRIC_RMSE_MODEL_X': 5.66, 'GEOMETRIC_RMSE_MODEL_Y': 5.495, 'IMAGE_DATE': '2019-11-14', 'IMAGE_QUALITY_OLI': 9, 'IMAGE_QUALITY_TIRS': 9, 'LANDSAT_ID': 'LC08_L1TP_122044_20191114_20191202_01_T1', 'LEVEL1_PRODUCTION_DATE': 1575301122000, 'NOMINAL_SCALE': 30, 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0', 'SATELLITE': 'LANDSAT_8', 'SENSING_TIME': '2019-11-14T02:52:28.2228010Z', 'SOLAR_AZIMUTH_ANGLE': 153.700989, 'SOLAR_ZENITH_ANGLE': 45.366386, 'SR_APP_VERSION': 'LaSRC_1.3.0', 'WRS_PATH': 122, 'WRS_ROW': 44, 'system:asset_size': '627.230926 MB', 'system:band_names': ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'], 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_122044_20191114', 'system:index': 'LC08_122044_20191114', 'system:time_end': '2019-11-14 02:52:28', 'system:time_start': '2019-11-14 02:52:28', 'system:version': 1576236040308713}


Next, examine the mean, median surface reflectance and/or NDVI layers we've visualized. Which one is better? Which band should we include into the classification?

We can also use the Shenzhen GEE app from the [previous chapter](4_DownloadLandsatFromGEE.ipynb) to examine the time series:

In [11]:
%matplotlib inline
from IPython.display import IFrame
IFrame('https://plewis.users.earthengine.app/view/shenzhen','100%',490)

In addition to switching layers on and off, adjusting opacity, we can also use python code to some simple mathmatical operations, e.g. calculating the differences:

In [42]:
mean_ndvi = mean.normalizedDifference(['NIR', 'Red'])
median_ndvi = median.normalizedDifference(['NIR', 'Red'])
median_ndwi = median.normalizedDifference(['Green','NIR'])

Map.addLayer(mean_ndvi.subtract(median_ndvi), {'min': -0.2,'max': 0.2}, 'Diff in NDVI')
Map.addLayer(median_ndwi, ndvi_colorized_vis, 'NDWI from Median LS')
Map

Map(bottom=57347.0, center=[22.634, 114.19], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

## K-means unsupervised classification with GEE

Let's start with the 2019 Median image as an example. In the following code cell, a function named `unsupervised_classifier` has been defined to classify an image by take five input parameters (or 'arguments'). You can don't have to worry about the detail of the code except the very first line, where information about the 'arguments' can be found.

In [46]:
import matplotlib.pyplot as plt
def unsupervised_classifier(image, aoi, year, n_clusters=5, output_filename='', DISPLAY_ON_MAP = False):
    '''This function provides a simple K-means classifier,
    with a default no. of cluster of 5. User will need to specify 
    an AOI and an image to be classified.
    Optional arguments:
    n_clusters defines the number of clusters in the K-means classifier;
    output_filename should be a quoted string, e.g. 'Shenzhen_Landsat_Kmeans_2019.tif';
    DISPLAY_ON_MAP can be switched on, so the cluster map will be added to Map.
    '''
    
    # Make the training dataset:
    training_points = image.sample(**{
        'region': aoi,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    Rcolors = ['#%02x%02x%02x' % tuple((int(c*254) for c in plt.cm.plasma(n)[:-1])) 
           for n in range(0,255,int(255/n_clusters))]

    #Map.addLayer(training_points, {}, 'training points', False) # No need to visualise this layer

    # Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training_points)

    # Cluster the input using the trained clusterer.
    class_result = image.cluster(clusterer)
    
    # Reclassify the map to avoid zero, in case of masking. E.g. from [0, 1, 2, 3, 4] to [1, 2, 3, 4, 5]
    #class_result = class_result.remap(list(range(0, n_clusters)), list(range(1, n_clusters+1)))

    if type(DISPLAY_ON_MAP) == geemap.geemap.Map:
        # Display the clusters with random colors.
        Map.addLayer(class_result, {'min': 0, 'max': n_clusters-1,'palette':Rcolors}, 'Kmeans Clusters '+str(year), opacity=1.0)
        legend_keys = ['Cluster {}'.format(n) for n in range(5)]
        Map.add_legend(legend_keys=legend_keys, legend_colors=Rcolors, position='bottomright')

    if output_filename == '':
        print(f'{year} classification finished. No output exported.')
    else:
        #Export the result directly to your computer/Hub:
        geemap.ee_export_image(class_result, filename=output_filename, \
                            scale=900, region=aoi, file_per_band=False) 
        # When scale is small, GEE won't allow downloading due to size limitation

    return class_result

The above cell only 'defines' a function but does NOT execute it, so we now need to actually run it by calling the function name with appropriate arguments:

In [48]:
class_result = unsupervised_classifier(mean, shenzhen_buffer, 2019, n_clusters=5, 
                                       DISPLAY_ON_MAP = Map)
Map

2019 classification finished. No output exported.


Map(bottom=228808.0, center=[22.478933900916914, 113.97937774658205], controls=(WidgetControl(options=['positi…

When we executed the `unsupervised_classifier` function, a variable `class_result` was used to received the returned values from the function, and we will use this `class_result` variable in the following code to inspect our classification output.

## Link unsupervised clusters into Land Use Land Cover (LULC) classes
You need to carefully compare the K-means results with the original images to decide which cluster belongs to what class. Then, you can name the classes and assign appropriate RGB colours accordingly. However, the classified clusters may differ between different years, so you will need to change and classification setting (i.e. `n_clusterss`) and the legend for some of the years. Also, bare in mind, there might be mis-classified pixels. How can you improve the results?

In [50]:
legend_keys = ['Urban', 'Vegetation', 'Bright Urban',  'Water', 'Shaded Vegetation']
legend_colors = ['#FFFFB3', '#8DD3C7', '#FFFFB0','#80B1D3', '#8DD3C0']

Map.addLayer(class_result, {'min': 0, 'max': 4, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

The number of clusters to use depends on the image, and it may different between different years. For the 2019 example, we used 5 clusters and the results seem to include two 'Urban' and two 'Vegetation' clusters. In such situation, we may consider grouping multiple clusters together, by re-mapping the cluster numbers:

In [51]:
# Reclassify the map. You may want to avoid zero, in case of masking.
remapped_class_result = class_result.remap([0, 1, 2, 3, 4], [1, 2, 1, 3, 2])

legend_keys = ['Urban', 'Vegetation', 'Water']
legend_colors = ['#FFFFB3', '#8DD3C7','#80B1D3']

Map.addLayer(remapped_class_result, {'min': 1, 'max': 3, 'palette': legend_colors}, 'Remapped clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

However, sometimes you may find the reclustered results less satisfactory, so consider carefully before executing the re-mapping code. 

After remapping the clusters, we have just three final classes now.  Would this be the same as if we ran the `unsupervised_classifier` function with `n_clusters = 3`? Have a try. 

## Cluster areas
Next, we need to find out the size of each cluster, i.e. how many pixels belong to each class, and the total size of each class (hint: how big is each pixel?)

In [17]:
def calculate_class_size(class_result, year, legend_keys, PRINT_STATS=False):
    n_clusters = len(legend_keys)
    
    #landsat_stats = geemap.image_stats(class_result, scale=90)
    #print(landsat_stats.getInfo())

    if year in [2000, 2004]: 
        print(f'Downscaling for year {year}')
        scale_factor= 3
    else: scale_factor= 1
    
    stats = {'Year': year}
    for i in range(n_clusters):
        remap = numpy.zeros(n_clusters)
        remap[i] = 1
        class_0 = class_result.remap(list(range(0, n_clusters)), list(remap))
        #print(list(remap))
        class_stats0 = geemap.image_stats(class_0, scale=30*scale_factor)
        #print(class_stats0.getInfo())
        
        sum_class0 = class_stats0.getInfo()['sum']
        sum_clean = int(sum_class0['remapped'])
        if PRINT_STATS:
            print(legend_keys[i], 'has', sum_clean, 'pixles.')
        stats[legend_keys[i]] = sum_clean * (scale_factor**2)
        
    return stats

In [18]:
legend_keys = ['Urban', 'Vegetation', 'Bright Urban',  'Water', 'Shaded Vegetation']        
stats_2019 = calculate_class_size(class_result, 2019, legend_keys, PRINT_STATS=True)

Urban has 1006156 pixles.
Vegetation has 821889 pixles.
Bright Urban has 374495 pixles.
Water has 901949 pixles.
Shaded Vegetation has 654431 pixles.


These are the numbers we will need for the R modeling. We will also show you how to export these values automatically later. But for now, have another close examination of the classification results and the functions we used to generated them. 

## Repeat for multiple years
By now, you should be able to understand that the `load_landsat_collection` function allows us to just vary the 'year' variable (and leave AOI and Cloud Tolerance constant) to loop through a time series of data. Similarly, for the `unsupervised_classifier` function, we can vary the input 'image' variable (an output from the `load_landsat_collection` function) in order to classify multiple images. Below is a loop built for running K-means classification for Landsat time series for the Shenzhen area. Years between 1987 to 2016 are selected to match our socio-economic data. When this cell is running (you can spot a star key on the LHS of the cell), you can track the progress by looking at the output messages. Once the cell finished running, an integer number will be shown outside of the LHS of the cell, and you can examine the layers in the Map (if DISPLAY_ON_MAP flag was set to True)

In [19]:
cluster_pixels = []

# Only to demonstrate every 10 years, but you should run every year, ideally in groups
for year in range(1986, 2017, 10): 
    
    median_image = load_landsat_collection(year, shenzhen_buffer, cloud_tolerance=3,\
                                          MEDIAN_ONLY = True)
    class_result = unsupervised_classifier(median_image, shenzhen_buffer, year,\
                    n_clusters=5, DISPLAY_ON_MAP = True) #output_filename=f'Shenzhen_Landsat_Kmeans_{year}.tif'
    
    ## -- IMPORTANT --  change length of list to match n_clusters
    legend_keys = ['Class 1', 'Class 2', 'Class 3',  'Class 4', 'Class 5'] ## change the class names
    
    stats = calculate_class_size(class_result, year, legend_keys)
    print(stats)
    stats_res = numpy.fromiter(stats.values(), dtype=int)
    

    cluster_pixels.append(stats_res)
#Map

1986 classification finished. No output exported.
{'Year': 1986, 'Class 1': 1295840, 'Class 2': 1137617, 'Class 3': 397936, 'Class 4': 679908, 'Class 5': 247620}
1996 classification finished. No output exported.
{'Year': 1996, 'Class 1': 1252350, 'Class 2': 1064788, 'Class 3': 382639, 'Class 4': 551019, 'Class 5': 508124}
2006 classification finished. No output exported.
{'Year': 2006, 'Class 1': 934998, 'Class 2': 830316, 'Class 3': 362536, 'Class 4': 860025, 'Class 5': 771044}
2016 classification finished. No output exported.
{'Year': 2016, 'Class 1': 620490, 'Class 2': 740670, 'Class 3': 488368, 'Class 4': 792561, 'Class 5': 1116831}


After running the above loop of code to classify a time series, all cluster areas (i.e. pixel counts) have been recorded into the `cluster_pixels` variable, and cluster layers are mapped into the Map.

## Export results to excel (.csv) files
Lastly, we will use the following code to export the sizes of each class/cluster into a csv file, and you will need this file for R modeling in the next part of the coursework.

In [128]:
### saving the results. Do this even if the processing loop crashes.
### The results may well be in the memory and can be saved. 
### When processing in batches, make sure you change the filename so that it doesn't get overwrtten for the next batch 
header = 'Year, ' + ', '.join( legend_keys )
numpy.savetxt("Shenzhen_pixel_stats.csv", cluster_pixels, delimiter=",", header=header,fmt='%i')       

**Congratulations (on finishing the classification part)!**

So far, we have processed the entire time series of Landsat data without actually downloading any of them to our local PC (everything is done on the Cloud!), except the .CSV file. I hope you've enjoyed playing with GEE here. Now you should be able to open the CSV file in excel or R to continue with the [modeling part](https://github.com/qwu-570/GEOG0027_Coursework/blob/2020-2021/docs/6_UrbanModel.ipynb) of the coursework.

## STOP NOW and check you have got all the previous code working and you know what it is doing
## ADVANCED TASKS

Below you will find some extra code for further advanced classification techniques. These are harder to use and will require some concentration and consideration of what the code is asking GEE to do. The following tasks are not strictly neccessary for the coursework, but are satisfying, and once complete, may allow for easier analysis and use within the modelling section.

## Using ENVI ROIs for ground truthing
If you have previously saved a collection of ROIS as a .csv file in ENVI, we can use that file to create training data for GEE.
GEE likes to get training data as a series of points. These can be extracted from the ENVI file using a module to be downloaded. Use a terminal in jupyterhub to donwload the module using the following command and make sure it is in the same directory as this notebook.

`wget https://github.com/UCL-EO/GEOG0027_Coursework/blob/main/docs/ENVI_ROI.py'

We will use a function that will load information we need from a saved collection of ROIs created in ENVI. To call it you need to tell it where to find the .csv file with the ROIs in it. There is an example file given, with 3 classes set from 1986. You will need your own file for the coursework.

To make a successful classification, make sure the ROIs are valid for all time points you wish to classify. As the Shenzhen region undergoes many changes since 1984, you may wish to create seperate ROI files for the full time series. This decision is up to you.

In [2]:
from ENVI_ROI import get_ENVI_ROIS
roi_file = '1986_3type_ROIs.csv' ### original file, it works, but not very well
ROIs = get_ENVI_ROIS(roi_file,points_per_region=100) 
### points_per_region is the number of subsamples to use to send to GEE. 
### More points send more training data

Successfully loaded 3 ROIs
All_water total 4 subregions
All_veg total 3 subregions
Urban total 1 subregions


The following code will plot the points from the ROI file to the map. 
Check they are in the location you expect them to be in?
Do the ROI regions created for an image from 1986 align well with 2019?

In [54]:
#### add the points taken from the ENVI ROIs to the map
for roi in ROIs['ROI']:
    visOptions = {'color':roi['color'],'pointRadius':1}
    Map.addLayer(roi['Points'],visOptions,roi['name'])
Map

Map(bottom=228568.0, center=[22.63112385675556, 114.12151336669923], controls=(WidgetControl(options=['positio…

## Testing the classification accuracy (ADVANCED)
Another task we can perform using the ROIs from ENVI is to check whether a classification is accurate.
This is a valid technique for the previous unsupervised classification. Why does it not tell us much about the supervised classification using the code below? How can you independently asses this classification? (Hint: you'll need to copy some code cells, and have another file from ENVI)

In [60]:
def compute_errors(class_result, ROIs):
    '''This function provides a simple error matrix using
    the ROIS loaded previously from an ENVI file.
    '''
    trainingFeatures = ee.FeatureCollection([roi['Points'] for roi in ROIs['ROI']]).flatten()
    class_points= class_result.reduceRegions(collection=trainingFeatures,reducer=ee.Reducer.median(),
                                             scale= 500,crs= 'EPSG:5070')
    return class_points.errorMatrix('class','median').getInfo()

The following function will perform the supervised Minimum Distance classification algorithm using the points taken from the ROI file.

In [64]:
### pixel error matrix for the unsupervied classification
compute_errors(class_result,ROIs)

[[17, 0, 5, 278, 0],
 [0, 114, 0, 0, 186],
 [99, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [55]:
def supervised_classifier(image, ROIs, year, output_filename='', DISPLAY_ON_MAP = False):
    '''This function provides a simple minimum distance classifier,
    using the ROIS loaded previously from an ENVI file.
    Optional arguments:
    output_filename should be a quoted string, e.g. 'Shenzhen_Landsat_Kmeans_2019.tif';
    DISPLAY_ON_MAP can be switched on, so the cluster map will be added to Map.
    '''
    
    # Make the training dataset:
    trainingFeatures = ee.FeatureCollection([roi['Points'] for roi in ROIs['ROI']]).flatten()
    classifierTraining = image.sampleRegions(collection= trainingFeatures,
            properties= ['class'],
            scale= 30
        )
    
    # Instantiate the clusterer and train it.
    bands=['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
    # Option for independent work here: can you change the 'minimumDistance' to another algorithm?
    # You'll need to get internet searching for the GEE options to try
    classifier = ee.Classifier.minimumDistance().train(
        features= classifierTraining,
        classProperty= 'class',
        inputProperties= bands
    )

    # Cluster the input using the trained clusterer.
    class_result = image.classify(classifier)
    
    if type(DISPLAY_ON_MAP) == geemap.geemap.Map:
        # Display the clusters with ROI set colors.
        Map.addLayer(class_result, {'min': 0, 'max': ROIs['n_ROI']-1, 'palette': [roi['color'] for roi in ROIs['ROI']]},
                    'MinDist '+str(year))

    if output_filename == '':
        print(f'{year} classification finished. No output exported.')
    else:
        #Export the result directly to your computer/Hub:
        geemap.ee_export_image(class_result, filename=output_filename, \
                            scale=900, region=aoi, file_per_band=False) 
        # When scale is small, GEE won't allow downloading due to size limitation

    return class_result

Then we call the classifier and view it on the map. 
Check the layers to see if the classifier has matched the training data.

In [58]:
# Complete the supervised classification for the given year
classified_image = supervised_classifier(median,ROIs,2019,DISPLAY_ON_MAP=Map)

Map

## Testing the classification accuracy (ADVANCED)
Another task we can perform using the ROIs from ENVI is to check whether a classification is accurate.
This is a valid technique for the previous unsupervised classification. Why does it not tell us much about the supervised classification using the code below? How can you independently asses this classification? (Hint: you'll need to copy some code cells, and have another file from ENVI)

In [63]:
### pixel error matrix for the supervised classification
compute_errors(classified_image,ROIs)

[[278, 0, 22], [0, 300, 0], [0, 1, 99]]

## Time series supervised classification (ADVANCED)
Here is an example loop that will perform a supervised classifcation for multiple years. If you want to use this method, then some work will be required.

Can you make sure the supervised classification is performing correctly for each step of the loop?
Can you compute the error matrix (independently) for each step of the loop?

Can you modify the unsupervised classification loop to add the error matrix entries too?
Saving the error matrix for the unsupervised classifier will allow for manual editting of the saved results .csv file. This is sometimes required prior to modelling so that the columns match from year to year (clusters can switch order).

In [19]:
cluster_pixels = []

# Only to demonstrate every 10 years, but you should run every year, ideally in groups
for year in range(1986, 2017, 10): 
    
    median_image = load_landsat_collection(year, shenzhen_buffer, cloud_tolerance=3,\
                                          MEDIAN_ONLY = True)
    
    class_result = supervised_classifier(median,ROIs,year,DISPLAY_ON_MAP=False)
    
    legend_keys = [roi['name'] for roi in ROIs['ROI']]
    
    stats = calculate_class_size(class_result, year, legend_keys)
    print(stats)
    stats_res = numpy.fromiter(stats.values(), dtype=int)
    ### these next lines will also compute an error matrix for the results
    ### To make them work, you'll need to supply an alternative ROIs from an ENVI file 
    ### These lines can also be used to add error matrix information to the unsupervised 
    ### classification loop above too
    ### ------ CODE BLOCK ----- START
#     error_matrix = compute_errors(class_result,ROIs2)
#     print(error_matrix)
#     stats_res = numpy.hstack([stats_res,np.array(error_matrix).flatten()])
#     legend_keys = legend_keys+['EM_{}{}'.format(m,n) for m in range(len(error_matrix)) 
#                                                      for n in range(len(error_matrix))]
    ### ------ CODE BLOCK ----- END

    cluster_pixels.append(stats_res)
#Map

1986 classification finished. No output exported.
{'Year': 1986, 'Class 1': 1295840, 'Class 2': 1137617, 'Class 3': 397936, 'Class 4': 679908, 'Class 5': 247620}
1996 classification finished. No output exported.
{'Year': 1996, 'Class 1': 1252350, 'Class 2': 1064788, 'Class 3': 382639, 'Class 4': 551019, 'Class 5': 508124}
2006 classification finished. No output exported.
{'Year': 2006, 'Class 1': 934998, 'Class 2': 830316, 'Class 3': 362536, 'Class 4': 860025, 'Class 5': 771044}
2016 classification finished. No output exported.
{'Year': 2016, 'Class 1': 620490, 'Class 2': 740670, 'Class 3': 488368, 'Class 4': 792561, 'Class 5': 1116831}


In [141]:
error_matrix = [[17, 0, 5, 278, 0],
 [0, 114, 0, 0, 186],
 [99, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]
error_matrix = [[278, 0, 22], [0, 300, 0], [0, 1, 99]]

In [142]:
stats = {'Year': 2016, 'Class 1': 620490, 'Class 2': 740670, 'Class 3': 488368}#,
#          'Class 4': 792561, 'Class 5': 1116831}#,'error_matrix':error_matrix}

In [143]:
stats_res = numpy.fromiter(stats.values(),dtype=int)

In [144]:
stats_res = numpy.hstack([stats_res,np.array(error_matrix).flatten()])

In [145]:
legend_keys = [roi['name'] for roi in ROIs['ROI']]
legend_keys = legend_keys+['EM_{}{}'.format(m,n) for m in range(len(error_matrix)) 
                                                 for n in range(len(error_matrix))]

In [146]:
cluster_pixels = []
cluster_pixels.append(stats_res)

In [147]:
### saving the results. Do this even if the processing loop crashes.
### The results may well be in the memory and can be saved. 
### When processing in batches, make sure you change the filename so that it doesn't get overwrtten for the next batch 
header = 'Year, ' + ', '.join( legend_keys )
numpy.savetxt("test_pixel_stats.csv", cluster_pixels, delimiter=",", header=header,fmt='%i')       